# Libraries

In [1]:
import pyedflib
import plotly.express as px
from pathlib import Path
import os
import numpy as np
from scipy.signal import butter, filtfilt

# EDF class

In [18]:
class EDFFile:
    def __init__(self, file: Path):
        self.file_path = file.absolute().as_posix()
        self.file_name = file.name
        self.file_handle = pyedflib.EdfReader(self.file_path)
        self.file_header = self.file_handle.getHeader()
        self.signal_headers = None
        self.num_signals = self.file_handle.signals_in_file
        self.sample_frequency = None
        self.signals = []
        self.annotations = []
        
        self.load_raw_signals()
        self.load_signal_headers()
        self.load_annotations()
        self.load_sample_frequency()
        
        self.close()
        
    def load_raw_signals(self):
        
        for i in range(self.num_signals):
            signal_buffer = self.file_handle.readSignal(i)
            self.signals.append(signal_buffer)
    
    def load_signal_headers(self):
        if self.file_handle is None:
            self.file_handle = pyedflib.EdfReader(self.file_path)

        self.signal_headers = self.file_handle.getSignalHeaders()

    def load_annotations(self):
        if self.file_handle is None:
            self.file_handle = pyedflib.EdfReader(self.file_path)
        
        self.annotations = self.file_handle.readAnnotations()

    def load_sample_frequency(self):
        if self.file_handle is None:
            self.file_handle = pyedflib.EdfReader(self.file_path)
        
        self.sample_frequency = self.file_handle.getSampleFrequencies()[0]
        
        return self.sample_frequency
    
    def save_edf_filtered(self, output_path: Path, signals: np.ndarray):
        # Create a new EDF file to write the modified signals into
        new_file = pyedflib.EdfWriter(output_path / f'{self.file_name}.edf', n_channels=self.num_signals, file_type=pyedflib.FILETYPE_EDFPLUS)
        
        new_file.setHeader(self.file_handle.getHeader())
        
        for channel, signal in enumerate(signals):
            # Write the modified signal to the new file
            new_file.writeSamples(channel, signal)
            #writeSignal.send_signal(signal)
            
        new_file.close()
        
    def close(self):
        if self.file_handle is not None:
            self.file_handle.close()
            self.file_handle = None

In [19]:
def read_files_from_dir(directory: Path):
    extensions = ["edf", "bdf"]

    return [EDFFile(Path(file)) for file in os.scandir(directory) 
            if file.is_file() and file.name.endswith(tuple(extensions))]

# Filter design

In [20]:
def apply_butterworth_lowpass(signal, cutoff, fs, order=3):
    nyquist = 0.5 * fs
    normalized_cutoff = cutoff / nyquist
    b, a = butter(order, normalized_cutoff, btype='low', analog=False)
    filtered_signal = filtfilt(b, a, signal)
    
    return filtered_signal

def apply_notch_filter(signal, notch_freq, fs):
    nyquist = 0.5 * fs
    notch_normalized = notch_freq / nyquist
    b, a = butter(2, [notch_normalized - 0.02, notch_normalized + 0.02], btype='bandstop', analog=False)
    filtered_signal = filtfilt(b, a, signal)
    
    return filtered_signal

def apply_butterworth_filters(signal, cutoff, fs, order=3, notch_frequencies=None):
    # Apply low-pass filter
    filtered_signal = apply_butterworth_lowpass(signal, cutoff, fs, order)

    # Apply notch filters
    if notch_frequencies is not None:
        for notch_freq in notch_frequencies:
            filtered_signal = apply_notch_filter(filtered_signal, notch_freq, fs)

    return filtered_signal

In [21]:
directory = Path("data/edf/")
files = read_files_from_dir(directory)

In [14]:
files[-2].signals

[array([8.49616876, 8.51417533, 8.44415179, ..., 8.87725704, 8.89326089,
        8.90426093]),
 array([6.12751133, 6.12201817, 6.11863858, ..., 5.03062904, 5.03252983,
        5.03506541]),
 array([-3.33334525, -3.33335002, -3.33335479, ..., -3.33358248,
        -3.33357175, -3.33355506])]

# Visualization

In [ ]:
px.line(files[-2].signals[0][:300])

In [15]:
filtered_signals = apply_butterworth_filters(files[-2].signals[0], 15, files[-2].sample_frequency, 3, [50, 100, 150])

In [ ]:
px.line(filtered_signals[:300])

# Testing zone

## Save EDF file file.name + _filtered.edf

In [ ]:
from pyedflib import highlevel

In [ ]:
signal_headers = highlevel.make_signal_headers(channel_names, sample_frequency=fs)

header = highlevel.make_header(patientname='patient', gender='Female')

if highlevel.write_edf(f'{self.file_name}_filtered.edf', signals, signal_headers, header):
    print("The file has been successfully saved.")
else:
    print("Something went wrong with the saving process.")

In [ ]:
help(pyedflib.EdfWriter)

In [23]:
output_file_path = Path(r"data/filtered/")

In [24]:
files[-2].save_edf_filtered(output_file_path, filtered_signals)

OSError: file has already been opened

In [ ]:
for file in files:
    file.close()

In [ ]:
len(files[-2].signals)

In [ ]:
new_file = pyedflib.EdfWriter(r'data/filtered/test.edf', n_channels=len(files[-2].signals), file_type=pyedflib.FILETYPE_EDFPLUS)
        
new_file.setHeader(files[-2].file_handle.getHeader())

new_file.close()